<a href="https://colab.research.google.com/github/DingSu0123/-/blob/main/pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 79.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 75.7 MB/s eta 0:00:00


In [ ]:
pip install BayesianOptimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install jieba

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import zipfile
import os

In [ ]:
import torch
import datetime
import time

In [ ]:
import re
import nltk
import jieba
import pandas as pd
from nltk.corpus import stopwords
from model import TransferLearningBERT

In [ ]:
# 解压 punkt.zip 文件到 nltk_data/tokenizers 目录
with zipfile.ZipFile('/content/punkt.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/nltk_data/tokenizers')

# 将 nltk_data 目录添加到 nltk 的数据路径
nltk.data.path.append("/content/nltk_data")

In [ ]:
def read_preprocess_data(filename):
    df = pd.read_csv(filename, sep=r'\|-\|')
    df['text'] = df[['en_table_name', 'en_field_name', 'cn_table_name', 'cn_field_name', 'system_name']].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)
    df.dropna(subset=['text'], inplace=True)
    df['text'] = df['text'].str.lower()

    nltk.download('stopwords')
    stop_words = set(stopwords.words('english'))
    df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words]))

    df['text'] = df['text'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
    df['text'] = df['text'].apply(lambda x: re.sub(r'\s+', ' ', x))
    df['text'] = df['text'].apply(lambda x: x.strip())

    # 分词函数，用于对英文和中文进行分词
    def tokenize_text(text):
        # 分离中英文字符
        en_pattern = re.compile(r'[a-zA-Z]+')
        zh_pattern = re.compile(r'[\u4e00-\u9fa5]+')
        en_words = en_pattern.findall(text)
        zh_words = zh_pattern.findall(text)

        if len(en_words) > 0 and len(zh_words) > 0:
            en_tokens = nltk.word_tokenize(' '.join(en_words))
            zh_tokens = [w for w in jieba.cut(' '.join(zh_words)) if w.strip()]
            tokens = en_tokens + zh_tokens
        elif len(en_words) > 0:
            tokens = nltk.word_tokenize(text)
        else:
            tokens = [w for w in jieba.cut(text) if w.strip()]
        return ' '.join(tokens)

    # 对'text'列中的文本进行分词
    df['text'] = df['text'].apply(lambda x: tokenize_text(x))
    df['text'] = df['text'].apply(lambda x: x.replace('_', ' '))


    return df

In [ ]:
def predict_batch(texts, batch_size=1280):
    preds = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i: i+batch_size]

        encoded_dict = [model.tokenizer.encode_plus(text, add_special_tokens=True, max_length=128,
                                                    pad_to_max_length=True, return_attention_mask=True)
                        for text in batch_texts]

        input_ids = torch.tensor([enc['input_ids'] for enc in encoded_dict]).to(model.device)
        attention_masks = torch.tensor([enc['attention_mask'] for enc in encoded_dict]).to(model.device)

        with torch.no_grad():
            logits = model.model(input_ids, attention_mask=attention_masks)[0]

        predicted_labels = torch.argmax(logits, dim=1).cpu().numpy() + 1
        preds.extend(predicted_labels)

    return preds


In [ ]:
# 载入所需预测数据集
pred_data = read_preprocess_data('/content/附录2.csv')

<ipython-input-8-faea543c6180>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(filename, sep=r'\|-\|')
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# 加载模型
model = torch.load('/content/drive/MyDrive/Colab Notebooks/complete_model.pt')
model.model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
# 对每一行进行预测并保存预测结果和时间戳
texts = pred_data['text'].tolist()
start_time = time.time()  # 获取当前Unix时间戳
y_preds = predict_batch(texts)
end_time = time.time()

load_time = datetime.datetime.fromtimestamp(end_time).strftime('%Y-%m-%d %H:%M:%S')  # 转换成指定格式的日期时间字符串

pred_data['y_pred'] = y_preds
pred_data['load_time'] = load_time

pred_data.to_csv('prediction_results_new.csv', index=False)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
